# Monte Carlo Simulation for Random Coefficient Logit Model
Date: 10/17/2023  
Author: Hwikook Choe

In the industrial organization literature, random coefficient logit (mixed logit). One of papers is Revelt and Train (1998), which used panel data that each individual can make repeated choices. The author provides Matlab code in https://eml.berkeley.edu/Software/abstracts/train1006mxlmsl.html.

I wrote the Python code that use the same model from the above paper. The estimation speed is much faster than Matlab code (around 5-10 times), by utilizing the features of modern programming languages (JIT compilation and parallelization with Numba, fast random number generation with PyTorch).

# Part 0. Preparation

In [1]:
print('Hello World!')

Hello World!


## 0.1. Import Packages

In [2]:
import os
import gc
from time import time
import datetime

import pandas as pd
import polars as pl
import numpy as np
import scipy as sp
from numba import njit, prange
from numba.typed import List

import torch

from scipy.optimize import minimize, Bounds
from knitro.scipy import kn_minimize

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

np.float = np.float64

# Part 1. Model

## 1.1. Define Random Number Generator

In [4]:
def rand_halton_generator(beta_mean_length, data_length, n_draws):
    np.random.seed(0)
    halton_sampler = sp.stats.qmc.Halton(
        d=beta_mean_length,
        scramble=True,
        seed=0,
    )
    t1 = np.random.uniform(
        size=(
            data_length,
            beta_mean_length,
            n_draws,
        )
    )
    R = List()
    for i in range(data_length):
        t2 = halton_sampler.random(n_draws).transpose()
        t3 = np.remainder(t1[i,:,:] + t2, 1.)
        t4 = sp.stats.norm.ppf(t3)
        R.append(t4)
    
    return R

def rand_sobol_generator(beta_mean_length, data_length, n_draws):
    torch.manual_seed(0)
    soboleng = torch.quasirandom.SobolEngine(dimension=beta_mean_length, scramble=True, seed=0)
    t1 = torch.rand(size=(data_length, beta_mean_length, n_draws), dtype=torch.float64)
    R = List()
    for i in range(data_length):
        t2 = soboleng.draw(n_draws).transpose(0, 1)
        t3 = torch.remainder(t1[i, :, :] + t2, 1.)
        t4 = torch.distributions.Normal(0, 1).icdf(t3)
        R.append(t4.numpy())
    return R

@njit
def create_beta_random_list(halton_draws, beta_mean, beta_var_diag, data_length):
    n_draws = halton_draws[0].shape[1]
    # beta_var_mat_lower_triangle = np.diag(np.exp(beta_var_diag))
    beta_var_mat_lower_triangle = np.diag(beta_var_diag)
    R = List()
    
    for i in range(data_length):
        R.append(beta_mean.repeat(n_draws).reshape((-1, n_draws)) + beta_var_mat_lower_triangle @ halton_draws[i])
        
    return R

## 1.2. Define Model

In [5]:
@njit
def individual_choice_probability(
    beta, 
    x_matrix,
):
    denominator_base = np.exp(x_matrix @ beta)
    # denominator = 1 + denominator_base.sum(axis=0)
    denominator = denominator_base.sum(axis=0)
    choice_probability = denominator_base / denominator
    return choice_probability

@njit
def individual_trip_likelihood(
    beta,
    y_vector,
    x_matrix,
):
    full_choice_probability = individual_choice_probability(
        beta,
        x_matrix,
    )
    actual_choice_probability = full_choice_probability.transpose() @ y_vector
    return actual_choice_probability

@njit
def individual_likelihood_temp(
    beta,
    individual_data_y,
    individual_data_x,
):
    individual_data_x_length = len(individual_data_x)
    actual_choice_probability = np.ones(beta.shape[1])
    for i in range(individual_data_x_length):
        actual_choice_probability = actual_choice_probability * individual_trip_likelihood(
                beta,
                individual_data_y[i],
                individual_data_x[i],
            )
    return actual_choice_probability

@njit
def individual_likelihood(
    beta,
    individual_data_y,
    individual_data_x,
):
    return np.log(
        individual_likelihood_temp(
            beta,
            individual_data_y,
            individual_data_x,
        ).sum() / beta.shape[1]
    )

@njit(parallel=True)
def log_likelihood(
    beta_mean, 
    beta_var_diag, 
    beta_nonrandom, 
    data_y,
    data_x, 
    halton_draws
):
    n_draws = halton_draws[0].shape[1]
    data_length = len(data_x)
    beta_random_list = create_beta_random_list(
        halton_draws,
        beta_mean,
        beta_var_diag,
        data_length,
    )
    R = np.zeros(data_length)
    
    for i in prange(data_length):
        R[i] = individual_likelihood(
            np.concatenate(
                (
                    beta_random_list[i],
                    beta_nonrandom.repeat(
                        n_draws
                    ).reshape(
                        (-1, n_draws)
                    ),
                ),
                0,
            ),
            data_y[i],
            data_x[i],
        )
    return -R.sum()

In [6]:
@njit
def individual_trip_gradient(
    individual_beta,
    individual_halton_draws,
    y_vector,
    x_matrix,
    random_size,
    nonrandom_size,
    n_draws,
):
    total_size = 2 * random_size + nonrandom_size
    full_choice_probability = individual_choice_probability(individual_beta, x_matrix)
    prob_part = y_vector.repeat(n_draws).reshape((-1, n_draws)) - full_choice_probability
    
    t1_1 = x_matrix[:, :random_size].repeat(n_draws).reshape((x_matrix.shape[0], random_size, n_draws))
    t1_2 = individual_halton_draws * t1_1
    t1_3 = x_matrix[:, random_size:].repeat(n_draws).reshape((x_matrix.shape[0], nonrandom_size, n_draws))
    t1 = np.concatenate(
        (t1_1, t1_2, t1_3),
        axis=1,
    )
    
    R = np.zeros((total_size, n_draws))
    for i in range(total_size):
        R[i, :] = (prob_part * t1[:, i, :]).sum(axis=0)
    
    return R

@njit
def individual_gradient(
    individual_beta,
    individual_halton_draws,
    individual_data_y,
    individual_data_x,
    random_size,
    nonrandom_size,
    n_draws,
):
    individual_data_x_length = len(individual_data_x)
    R = np.zeros(
        (
            individual_data_x_length,
            2 * random_size + nonrandom_size,
            n_draws,
        )
    )
    for i in range(individual_data_x_length):
        R[i, :, :] = individual_trip_gradient(
            individual_beta,
            individual_halton_draws,
            individual_data_y[i],
            individual_data_x[i],
            random_size,
            nonrandom_size,
            n_draws,
        )
    indiv_likeli_value = individual_likelihood_temp(
        individual_beta,
        individual_data_y,
        individual_data_x,
    )
    return_value = (R.sum(axis=0) @ indiv_likeli_value) / indiv_likeli_value.sum()
        
    return return_value
    

@njit(parallel=True)
def gradient(
    beta_mean, 
    beta_var_diag, 
    beta_nonrandom, 
    data_y, 
    data_x, 
    halton_draws
):
    n_draws = halton_draws[0].shape[1]
    data_length = len(data_x)
    beta_random_list = create_beta_random_list(
        halton_draws,
        beta_mean,
        beta_var_diag,
        data_length,
    )
    R = np.zeros(
        (
            data_length,
            beta_mean.shape[0] + beta_var_diag.shape[0] + beta_nonrandom.shape[0]
        )
    )
    for i in prange(data_length):
        R[i, :] = individual_gradient(
            np.concatenate(
                (
                    beta_random_list[i],
                    beta_nonrandom.repeat(
                        n_draws
                    ).reshape(
                        (-1, n_draws)
                    ),
                ),
                0,
            ),
            halton_draws[i],
            data_y[i],
            data_x[i],
            beta_mean.shape[0],
            beta_nonrandom.shape[0],
            n_draws,
        )
    # return -R.sum(axis=0) / n_draws
    return -R.sum(axis=0)

In [7]:
@njit
def individual_trip_hessian(
    individual_beta,
    individual_halton_draws,
    y_vector,
    x_matrix,
    random_size,
    nonrandom_size,
    n_draws,
):
    total_size = 2 * random_size + nonrandom_size
    choice_index = np.argwhere(y_vector)[0][0]
    Pir = individual_choice_probability(individual_beta, x_matrix)
    
    # partial_Pir_betaa_all
    t1_1 = x_matrix[:, :random_size].repeat(n_draws).reshape((x_matrix.shape[0], random_size, n_draws))
    t1_2 = individual_halton_draws * t1_1
    t1_3 = x_matrix[:, random_size:].repeat(n_draws).reshape((x_matrix.shape[0], nonrandom_size, n_draws))
    t1 = np.concatenate(
        (t1_1, t1_2, t1_3),
        axis=1,
    ).transpose(0, 2, 1)
    t2_1 = (t1 * Pir.repeat(total_size).reshape((x_matrix.shape[0], n_draws, -1)))
    t2_2 = t2_1.sum(axis=0)
    t2_3 = Pir.repeat(total_size).reshape((x_matrix.shape[0], n_draws, -1)) \
        * (t2_2.transpose().repeat(x_matrix.shape[0]).reshape((total_size, n_draws, -1)).transpose())
    partial_Pir_betaa_all = t2_1 - t2_3

    part1_1 = y_vector.repeat(n_draws).reshape((-1, n_draws)) - Pir
    part1 = (part1_1.repeat(total_size).reshape((-1, n_draws, total_size)) * t1).sum(axis=0)
    
    part3 = np.zeros((n_draws, total_size, total_size))
    for i in range(n_draws):
        for j in range(x_matrix.shape[0]):
            part3[i, :, :] = part3[i, :, :] + np.outer(partial_Pir_betaa_all[j, i, :], t1[j, i, :])
    
    partial_Pir_betaa = partial_Pir_betaa_all[choice_index, :, :]
    
    return partial_Pir_betaa, part1, part3

@njit
def individual_hessian(
    individual_beta,
    individual_halton_draws,
    individual_data_y,
    individual_data_x,
    random_size,
    nonrandom_size,
    n_draws,
):
    total_size = 2 * random_size + nonrandom_size
    individual_data_x_length = len(individual_data_x)
    
    Ptr = np.zeros((individual_data_x_length, n_draws))
    for i in range(individual_data_x_length):
        Ptr[i, :] = individual_trip_likelihood(
                individual_beta,
                individual_data_y[i],
                individual_data_x[i],
            )
    Pr = np.zeros(n_draws)
        
    partial_Pir_betaa = np.zeros((individual_data_x_length, n_draws, total_size))
    part1 = np.zeros((n_draws, total_size))
    part3 = np.zeros((n_draws, total_size, total_size))
    for i in range(individual_data_x_length):
        partial_Pir_betaa[i, :, :], t1, t2 = individual_trip_hessian(
            individual_beta,
            individual_halton_draws,
            individual_data_y[i],
            individual_data_x[i],
            random_size,
            nonrandom_size,
            n_draws,
        )
        part1 = part1 + t1
        part3 = part3 + t2

    for i in range(n_draws):
        Pr[i] = np.prod(Ptr[:, i])
    
    partial_Pr_betaa = (
        partial_Pir_betaa / Ptr.repeat(total_size).reshape((individual_data_x_length, n_draws, total_size))
    ).sum(axis=0) * Pr.repeat(total_size).reshape((n_draws, total_size))

    p2 = np.zeros((total_size, total_size))
    for i in range(n_draws):
        p2 = p2 + np.outer(partial_Pr_betaa[i, :], part1[i, :]) - Pr[i] * part3[i, :, :]
    P_total = Pr.sum() / n_draws
    p2 = p2 / (n_draws * P_total)
    
    partial_oneoverP_betaa = - partial_Pr_betaa.sum(axis=0) / (n_draws * (P_total ** 2))
    
    p1 = np.outer(
        partial_oneoverP_betaa,
        (Pr @ part1) / n_draws
    )
    
    return p1 + p2

@njit(parallel=True)
def hessian(
    beta_mean, 
    beta_var_diag, 
    beta_nonrandom, 
    data_y, 
    data_x, 
    halton_draws,
):
    n_draws = halton_draws[0].shape[1]
    data_length = len(data_x)
    beta_random_list = create_beta_random_list(
        halton_draws,
        beta_mean,
        beta_var_diag,
        data_length,
    )
    R = np.zeros(
        (
            data_length,
            beta_mean.shape[0] + beta_var_diag.shape[0] + beta_nonrandom.shape[0],
            beta_mean.shape[0] + beta_var_diag.shape[0] + beta_nonrandom.shape[0],
        )
    )
    for i in prange(data_length):
        R[i, :, ] = individual_hessian(
            np.concatenate(
                (
                    beta_random_list[i],
                    beta_nonrandom.repeat(
                        n_draws
                    ).reshape(
                        (-1, n_draws)
                    ),
                ),
                0,
            ),
            halton_draws[i],
            data_y[i],
            data_x[i],
            beta_mean.shape[0],
            beta_nonrandom.shape[0],
            n_draws,
        )    
    return -R.sum(axis=0)

In [8]:
@njit
def BHHH_temp(
    individual_beta,
    individual_halton_draws,
    individual_data_y,
    individual_data_x,
    random_size,
    nonrandom_size,
    n_draws,
):
    t1 = individual_gradient(
        individual_beta,
        individual_halton_draws,
        individual_data_y,
        individual_data_x,
        random_size,
        nonrandom_size,
        n_draws,
    )
    return np.outer(t1, t1)

@njit(parallel=True)
def hessian_BHHH(
    beta_mean, 
    beta_var_diag, 
    beta_nonrandom, 
    data_y,
    data_x, 
    halton_draws
):
    n_draws = halton_draws[0].shape[1]
    data_length = len(data_y)
    beta_random_list = create_beta_random_list(
        halton_draws,
        beta_mean,
        beta_var_diag,
        data_length,
    )
    R = np.zeros(
        (
            data_length,
            beta_mean.shape[0] + beta_var_diag.shape[0] + beta_nonrandom.shape[0],
            beta_mean.shape[0] + beta_var_diag.shape[0] + beta_nonrandom.shape[0],
        )
    )
    for i in prange(data_length):
        R[i, :, :] = BHHH_temp(
            np.concatenate(
                (
                    beta_random_list[i],
                    beta_nonrandom.repeat(
                        n_draws
                    ).reshape(
                        (-1, n_draws)
                    ),
                ),
                0,
            ),
            halton_draws[i],
            data_y[i],
            data_x[i],
            beta_mean.shape[0],
            beta_nonrandom.shape[0],
            n_draws,
        )
    return R.sum(axis=0)



In [9]:
def LL(param):
    beta_mean = param[:random_size]
    beta_var_diag = param[random_size:2*random_size]
    beta_nonrandom = param[2*random_size:]
    return np.array(log_likelihood(beta_mean, beta_var_diag, beta_nonrandom, data_convert_y, data_convert_x, halton_draws))

def gradient_LL(param):
    beta_mean = param[:random_size]
    beta_var_diag = param[random_size:2*random_size]
    beta_nonrandom = param[2*random_size:]
    return gradient(
        beta_mean, 
        beta_var_diag, 
        beta_nonrandom, 
        data_convert_y,
        data_convert_x, 
        halton_draws,
    )

def hessian_LL(param):
    beta_mean = param[:random_size]
    beta_var_diag = param[random_size:2*random_size]
    beta_nonrandom = param[2*random_size:]
    return hessian(
        beta_mean, 
        beta_var_diag, 
        beta_nonrandom, 
        data_convert_y,
        data_convert_x, 
        halton_draws,
    )

def numerical_hessian_LL(param):
    temp_val = gradient_LL(param)
    dx = 1e-6
    v1 = []
    for i in range(param.shape[0]):
        # param_temp_diff1 = param.copy()
        # param_temp_diff1[i] += dx
        param_temp_diff2 = param.copy()
        param_temp_diff2[i] -= dx

        # v1.append((gradient_LL(param_temp_diff1) - gradient_LL(param_temp_diff2)) / (2 * dx))
        v1.append((temp_val - gradient_LL(param_temp_diff2)) / dx)

    return np.array(v1)

def hessian_BHHH_LL(param):
    beta_mean = param[:12]
    beta_var_diag = param[12:24]
    beta_nonrandom = param[24:]
    return hessian_BHHH(
        beta_mean, 
        beta_var_diag, 
        beta_nonrandom, 
        data_convert_y,
        data_convert_x, 
        halton_draws,
    )


# Part 2. Simulation Preparation

## 2.1. Set up Parameters

In [10]:
## ORIGINAL PARAMETER THAT RAN ON SERVER

n_households = 100
n_max_trips = 5
n_max_products = 5
n_min_items_per_trip = 2
n_max_items_per_trip = 6
n_simulation_draws = 5000

np.random.seed(23)

beta_random_mean = np.array(
    [
        -2., 1., -3., # coefficient for mean of price, promotion, price_promotion_interaction
        0.1, 0.2, 0.3, 0.4, 0.3, 0.2, 0.1, 0.0, -0.1, # coefficient for mean of size decile from 2 to 10
    ]
)
beta_random_std = np.array(
    [
        1.0, 1.0, 2.0, # coefficient for std of price, promotion, price_promotion_interaction
        1., 1., 1., 1., 1., 1., 1., 1., 1., # coefficient for std of size decile from 2 to 10
    ]
)
beta_nonrandom = np.random.uniform(
    low=-3.,
    high=3.,
    size = n_max_products - 1,
) # coefficient for brand-FE

## 2.2. Create Dataset

In [12]:
beta_individual = np.concatenate(
    [
        np.random.multivariate_normal(
            beta_random_mean, 
            np.diag(np.power(beta_random_std, 2)),
            size=n_households,
        ),
        beta_nonrandom.repeat(
            n_households
        ).reshape(
            (-1, n_households)
        ).transpose(),
    ],
    axis=1,
)

In [13]:
def data_generator(
    n_households,
    n_max_trips,
    n_max_products,
    n_min_items_per_trip,
    n_max_items_per_trip,
    beta_individual,
    seed_number = 42,
):
    '''
    types of random variables
    - price    : lognormal
    - promotion: Bernoulli with success probability 0.3
    - others   : uniform
    '''
    
    np.random.seed(seed_number)
    
    ## 1. household_code, trip_code
    
    total_df = pl.DataFrame(
        {
            'household_code': np.arange(n_households),
            'trip_code': [
                [*range(i)] for i in np.random.choice(
                    np.arange(1, n_max_trips + 1), 
                    n_households,
                )
            ],
        }
    ).explode(
        'trip_code',
    )
    total_size = total_df.shape[0]
    total_df = total_df.with_columns(
        pl.Series(name='idx', values=np.arange(total_size))
    )
    
    ## 2. item_code
    item_df = pl.DataFrame(
        {
            'item_code': [
                [*range(1, i+1)] for i in np.random.choice(
                    np.arange(n_min_items_per_trip, n_max_items_per_trip + 1),
                    total_df.shape[0],
                )
            ]
        }
    ).with_columns(
        pl.Series(name='idx', values=np.arange(total_size))
    )
    total_df = total_df.join(
        item_df,
        left_on='idx',
        right_on='idx',
    ).explode(
        'item_code',
    )
    total_size = total_df.shape[0]
    total_df = total_df.with_columns(
        pl.Series(name='idx', values=np.arange(total_size))
    )
    
    ## 3. price, promotion, price_promotion
    total_df = total_df.with_columns(
        [
            pl.Series(
                name='unit_price_cent',
                values=np.random.lognormal(
                    mean = 0.,
                    sigma = 1.,
                    size = total_size,
                )
            ),
            pl.Series(
                name='feature_or_display',
                values=np.random.choice(
                    2,
                    size=total_size,
                    p=[0.7, 0.3],
                )
            ),
        ]
    ).with_columns(
        (pl.col('unit_price_cent') * pl.col('feature_or_display')).alias('price_feat_disp')
    )
    
    ## 4. size_decile
    total_df = total_df.with_columns(
        pl.Series(
            name='size_decile',
            values=np.random.choice(
                np.arange(1, 11),
                size=total_size,
            )
        )
    )
    size_decile_temp = total_df['size_decile']
    
    ## 5. product_id
    total_df = total_df.with_columns(
        pl.Series(
            name='product_id',
            values=np.random.choice(
                n_max_products,
                size=total_size,
            )
        )
    )
    product_id_temp = total_df['product_id']
    
    ## 6. create dummy variable for size_decile, product_id
    total_df = total_df.to_dummies(
        columns=['size_decile', 'product_id']
    ).drop(
        columns=['size_decile_1', 'product_id_0']
    )

    selected_columns = [
        'unit_price_cent',
        'feature_or_display',
        'price_feat_disp',
    ] + [
        f'size_decile_{i}' for i in range(2, 11)
    ] + [i for i in total_df.columns if 'product_id' in i]
    total_df = total_df.select(
        [
            'household_code',
            'trip_code',
            'idx',
            'item_code',
        ] + selected_columns
    )
    
    ## 7. create outside option (all attributes are zero)
    df_temp = total_df.select(['household_code', 'trip_code']).unique(maintain_order=True)
    for i in total_df.columns[2:]:
        df_temp = df_temp.with_columns(
            pl.lit(0).alias(i)
        )
    total_df = pl.concat(
        [total_df, df_temp],
        how='vertical_relaxed',
    ).sort(
        by=['household_code','trip_code','item_code']
    )
    total_df = total_df.with_columns(
        pl.Series(
            name='idx',
            values=np.arange(total_df.shape[0])
        )
    )
    
    ## 8. purchased
    purchased_list = []
        
    for i in range(n_households):
        for j in total_df.filter(pl.col('household_code') == i)['trip_code'].unique(maintain_order=True):
            t1 = total_df.filter(
                (pl.col('household_code') == i) & (pl.col('trip_code') == j)
            ).select(selected_columns).to_numpy().astype(np.float64)
            t2 = individual_choice_probability(beta_individual[i, :], t1)
            t2 = t2 / t2.sum()
            t3 = np.random.choice(
                t2.shape[0],
                size=1,
                p=t2,
            )[0]
            t4 = np.zeros(t2.shape[0])
            t4[t3] = 1
            purchased_list.append(t4)
    purchased_list = np.concatenate(purchased_list, axis=None)
    total_df = total_df.with_columns(
        pl.Series(
            name='purchased',
            values=purchased_list,
        )
    )
        
    ## 9. hms_index
    hms_index_df = total_df.select(
        ['household_code', 'trip_code']
    ).unique(
        maintain_order=True
    )
    hms_index_df = hms_index_df.with_columns(
        pl.Series(
            name='hms_index',
            values=np.arange(hms_index_df.shape[0]),
        )
    )
    total_df = total_df.join(
        hms_index_df,
        on=['household_code', 'trip_code'],
        how='left',
    )
    
    ## 10. final
    # total_df = total_df.with_columns(
    #     [
    #         pl.Series(
    #             name='product_id_copy',
    #             values=product_id_temp,
    #         ),
    #         pl.Series(
    #             name='size_decile_copy',
    #             values=size_decile_temp,
    #         ),
    #     ]
    # )
    selected_columns = [
        'hms_index', 'household_code',
        'unit_price_cent', 'feature_or_display', 'price_feat_disp',
    ] + [
        i for i in total_df.columns if ('size_decile' in i) & (i != 'size_decile_copy')
    ] + [
        i for i in total_df.columns if ('product_id' in i) & (i != 'product_id_copy')
    ] + ['purchased']
    
    return total_df.select(selected_columns).to_pandas()

The following code creates the data file that can be run for Train's code.

In [14]:
# df = data_generator(
#     n_households,
#     n_max_trips,
#     n_max_products,
#     n_min_items_per_trip,
#     n_max_items_per_trip,
#     beta_individual,
#     seed_number=42,
# )

# t1 = df.columns[2:-1].tolist()
# df = df.loc[:, ['household_code', 'hms_index', 'purchased'] + t1]
# df.loc[:, 'household_code'] = df.loc[:, 'household_code'] + 1
# df.loc[:, 'hms_index'] = df.loc[:, 'hms_index'] + 1
# df.to_csv('Train_file.csv', index=False)

The following code creates the data (in the format of pandas DataFrame) that can be run for this code.

In [15]:
df = data_generator(
    n_households,
    n_max_trips,
    n_max_products,
    n_min_items_per_trip,
    n_max_items_per_trip,
    beta_individual,
    seed_number=42,
)

draw_household_restriction = df.loc[:, ['hms_index', 'household_code']].drop_duplicates()
selected_columns = ['hms_index',
    # 'size_quintile_2',
    # 'size_quintile_3',
    # 'size_quintile_4',
    # 'size_quintile_5',
    'unit_price_cent',
    'feature_or_display',
    'price_feat_disp',
    'size_decile_2',
    'size_decile_3',
    'size_decile_4',
    'size_decile_5',
    'size_decile_6',
    'size_decile_7',
    'size_decile_8',
    'size_decile_9',
    'size_decile_10',
] + [
    i for i in df.columns if ('product' in i) and (i not in ['product_id_copy', 'unique_product_index'])
    # 'product_id_others'
] + ['purchased']
df = df.loc[:, selected_columns].to_numpy()

data_convert_x = List(
    List(
        df[
            np.argwhere(df[:, 0] == i1).flatten(),
            1:-1,
        ] for i1 in draw_household_restriction.loc[draw_household_restriction.household_code == i0, 'hms_index']
    ) for i0 in draw_household_restriction.household_code.unique()
)

data_convert_y = List(
    List(
        df[
            np.argwhere(df[:, 0] == i1).flatten(),
            -1,
        ] for i1 in draw_household_restriction.loc[draw_household_restriction.household_code == i0, 'hms_index']
    ) for i0 in draw_household_restriction.household_code.unique()
)

del df
_ = gc.collect()
print(selected_columns)

['hms_index', 'unit_price_cent', 'feature_or_display', 'price_feat_disp', 'size_decile_2', 'size_decile_3', 'size_decile_4', 'size_decile_5', 'size_decile_6', 'size_decile_7', 'size_decile_8', 'size_decile_9', 'size_decile_10', 'product_id_1', 'product_id_2', 'product_id_3', 'product_id_4', 'purchased']


In [16]:
random_size = 12
nonrandom_size = len(selected_columns) - random_size - 2

# Part 3. Simulation

In [18]:
halton_draws = rand_sobol_generator(
    random_size,
    len(data_convert_x),
    n_simulation_draws,
    # 500, 
)

# param_temp = np.zeros(random_size * 2 + nonrandom_size)
# param_temp = np.array([-3.162890960619705, 1.2926769598408374, -4.559387179163292, 0.5910878786457772, 0.3937294919733346, 0.13754050357182887, 1.4630953394679025, 0.6154488025409559, -1.4434199243268253, -0.05689509044078872, 0.5320634290085866, 0.052435313793330385, 1.5179365975808612, 4.3533784332526195e-12, 2.8661667142399283, 3.8993759495076104, 2.778040810405738, 3.0196187676330326, 1.1086539805268745, 0.6480594272624226, 4.413992947995717, 1.5673725685087867, 2.719874498824228e-11, 2.267207931649311, 0.46411179678543313, 4.22120939977727, 2.01758301586564, -0.9041527695715422])
# param_temp = np.hstack(
#     (
#         np.zeros(random_size),
#         np.ones(random_size),
#         np.zeros(nonrandom_size)
#     )
# )

param_temp = np.array([-2.1080285188201473, 1.250500181805926, -3.021755197448159, 0.668742751085551, 0.5552750499106998, 0.7584979720481099, 0.7345330431164304, 0.48144569003802723, 0.12432023565038632, 0.2930271785392987, -0.4415526760276032, 0.002158664471362114, 1.0744245954593215, 2.8465778054502393e-11, 2.0312156894994344, 0.6476449188788029, 0.5303672772680743, 1.367868252382003, 1.381776723725218, 1.1054252443757806, 0.8362056654441385, 1.42981514598112, 1.9234595860438535, 1.3857802899294103, -0.11148629529231989, 2.534388108390893, 1.4905714989452725, -1.583390291840578])

In [20]:
%%time
t1 = LL(param_temp)
t1

CPU times: user 1.13 s, sys: 26.6 ms, total: 1.16 s
Wall time: 193 ms


array(1510.68346951)

In [40]:
%%time
t2 = gradient_LL(param_temp)
t2

CPU times: user 4.77 s, sys: 2.87 s, total: 7.64 s
Wall time: 466 ms


array([ 70.60109584,   2.35492952,  28.54269228,  -2.02999643,
        -1.22808047,  -2.32828232,  -6.72298114,   0.90099625,
         1.86476936,   1.91157036,   1.64043334,   2.22600702,
       -19.15053393,   1.35715573,   7.93143578,   0.95230872,
        -0.2463828 ,  -1.15976888,  -2.19082332,   2.06522394,
        -1.40668492,   1.518955  ,  -0.61839485,   1.68228391,
        10.26680977, -36.63325675,  -8.11969803,  20.78068825])

In [21]:
%%time
t3 = hessian_LL(param_temp)
t3

CPU times: user 35min 52s, sys: 1min 51s, total: 37min 43s
Wall time: 3min 30s


array([[ 1.30549480e+02,  5.54102359e+00,  1.01884670e+01,
         9.49295085e+00,  9.56501604e+00,  7.89991663e+00,
         8.58252834e+00,  8.53683654e+00,  6.45283957e+00,
         5.45010453e+00,  4.96530535e+00,  5.16752776e+00,
         7.14345505e+01, -5.35094562e-01,  1.08182236e+01,
         1.82871820e+00,  1.43943173e+00,  3.04708341e+00,
         3.94740955e+00,  2.87833968e+00,  2.19410387e+00,
         2.35800181e+00,  3.61422522e+00,  2.79209013e+00,
         1.03578977e+01,  2.71297120e+01,  2.12395140e+01,
         4.11371828e+00],
       [ 5.54102359e+00,  5.57398714e+01,  2.67171279e+01,
         4.07651474e+00,  3.46842191e+00,  2.68785299e+00,
         1.39032782e+00,  2.46471001e+00,  2.96712709e+00,
         3.16984718e+00,  9.13948726e-01,  3.59668078e+00,
        -7.59995579e+00, -3.90899646e-01,  8.61734478e+00,
         3.62475793e-01,  3.99148329e-01, -4.32964160e-01,
        -4.75109217e-01, -8.12758942e-01,  5.60610177e-01,
        -2.97302380e-01, -4.82

In [23]:
np.sqrt(np.diag(np.linalg.inv(t3)))

/var/tmp/ipykernel_29171/2286168747.py:1: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(np.diag(np.linalg.inv(t3)))


array([0.17150288, 0.27635818, 0.58187477, 0.21169597, 0.20222185,
       0.22995574, 0.2343225 , 0.2261269 , 0.2441433 , 0.25946396,
       0.35864996, 0.25454573, 0.1183396 ,        nan, 0.37053342,
       0.53131364, 0.66694726, 0.39988681, 0.40720656, 0.42565692,
       0.49308854, 0.40759645, 0.47647646, 0.42015276, 0.17303266,
       0.20095687, 0.16815076, 0.22668477])

In [80]:
bound_constraint = Bounds(
    [-np.inf] * random_size + [0] * random_size + [-np.inf] * nonrandom_size,
    [np.inf] * (2 * random_size + nonrandom_size)
)

Xi_history=[]
fval_history=[]

def callbackF(Xi):
    t1 = LL(Xi)
    time_append = time()
    Xi_history.append(Xi)
    fval_history.append(t1)
    
    print(f'current_fval: {t1}')
    
    if len(time_list) > 1:
        print(f'elapsed_time for this iteration {time_list[-1] - time_list[-2]:.2f}')
    

time_list = [time()]

opt_result = minimize(
    fun=LL,
    x0 = param_temp,
    jac=gradient_LL,
    # hess=hessian_LL,
    method=kn_minimize,
    bounds=bound_constraint,
    callback=callbackF,
    options={
        # 'hessopt': 2
        # 'numthreads': os.cpu_count(),
    },
)


           Academic License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 13.0.1



/home/hkchoe/.conda/envs/hkchoe_server7/lib/python3.9/site-packages/numpy/ctypeslib.py:518: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return asarray(obj)
/home/hkchoe/.conda/envs/hkchoe_server7/lib/python3.9/site-packages/numpy/ctypeslib.py:518: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return asarray(obj)


Knitro presolve eliminated 0 variables and 0 constraints.

concurrent_evals:        0
The problem is identified as bound constrained only.

Problem Characteristics                                 (   Presolved)
-----------------------
Objective goal:  Minimize
Objective type:  general
Number of variables:                                 28 (          28)
    bounded below only:                              12 (          12)
    bounded above only:                               0 (           0)
    bounded below and above:                          0 (           0)
    fixed:                                            0 (           0)
    free:                                            16 (          16)
Number of constraints:                                0 (           0)
    linear equalities:                                0 (           0)
    quadratic equalities:                             0 (           0)
    gen. nonlinear equalities:                        0 (           0)
    

/home/hkchoe/.conda/envs/hkchoe_server7/lib/python3.9/site-packages/numpy/ctypeslib.py:518: RuntimeWarning: A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.
  return asarray(obj)


current_fval: 2037.2970717634016
current_fval: 1909.1013705999721
current_fval: 1766.1845717955425
current_fval: 1705.2175551205614
current_fval: 1679.7688496733792
current_fval: 1634.662602692714
current_fval: 1626.6799689162867
current_fval: 1594.9707920752867
current_fval: 1585.9142285727803
      10    1.576533e+03   0.000e+00   2.270e+00   3.064e+00        0
current_fval: 1576.5326548282621
current_fval: 1553.0518582641268
current_fval: 1546.9126378205792
current_fval: 1540.7996997737946
current_fval: 1538.1773058471601
current_fval: 1537.9677247229622
current_fval: 1532.1405799288452
current_fval: 1527.0525819808158
current_fval: 1522.400772745035
current_fval: 1522.3770849660632
      20    1.521628e+03   0.000e+00   4.676e+00   3.372e+00        0
current_fval: 1521.6284713478756
current_fval: 1516.7169519323293
current_fval: 1513.748099832956
current_fval: 1513.1526238674996
current_fval: 1511.597782759123
current_fval: 1510.9077382613214
current_fval: 1510.760167781977
current

In [131]:
se = np.sqrt(np.linalg.inv(t3).diagonal())
z_score = param_temp / se
p_value = sp.stats.norm.sf(abs(z_score))*2
column_display_list = selected_columns[1:-1]

In [132]:
est_result_df = pd.DataFrame(
        {
            'name': [f'{i}_mean' for i in column_display_list[:random_size]] + [f'{i}_std' for i in column_display_list[:random_size]] + column_display_list[random_size:],
            'est' : param_temp.round(4),
            's.e.': se.round(4),
            'z'   : z_score.round(4),
            'p'   : p_value.round(4),
        }
    )